## Load Data

In [1]:
import json
import numpy as np
import pandas as pd
import os

In [2]:
path = 'json/'
filename = [] #json資料夾下所有檔案名稱(YYYY-MM-DD.json)
DF_list = []  #YYYY-MM-DD

for i in os.listdir(path):
    filename.append(i)
    DF_list.append(i[:-5])

In [3]:
day = input()
day_index = DF_list.index(day)

2018-10-02


In [4]:
#------------Stored testing data---------------
files_recent = {}
fr_list = []
count = 5

for i in range(day_index-count,day_index):
    fr_list.append(filename[i][:-5])
    with open(path+filename[i]) as f:
        symbol_f = json.load(f)
    files_recent[filename[i][:-5]] = symbol_f
    f.close

print(fr_list)

['2018-09-25', '2018-09-26', '2018-09-27', '2018-09-28', '2018-10-01']


In [5]:
print(files_recent['2018-10-01']['0050'].keys())

dict_keys(['adj_close', 'close', 'high', 'low', 'open', 'volume'])


In [20]:
Stock_sym = [sym for sym in files_recent[fr_list[-1]].keys()]
Stock_sym.remove('id')
New_sym = []
Stock_feature = []
Close = []
#Open = []
#Volume = []
#C = []
#O = []

for symbol in Stock_sym:
    tmp = []
    for i in range(0,count):
        if symbol not in files_recent[fr_list[-1-i]].keys():
            break
        for attr in ['adj_close','close','high','low','open','volume']: #['adj_close','close','high','low','open','volume']:
            if files_recent[fr_list[-1-i]][symbol][attr] == None or files_recent[fr_list[-1-i]][symbol][attr] == 'NULL':
                break
            #if attr == 'close':
            #    C.append(float(files_recent[fr_list[-1-i]][symbol]['close']))
            #if attr == 'open':
            #    O.append(float(files_recent[fr_list[-1-i]][symbol]['open']))
            if attr != 'close':
                tmp.append(float(files_recent[fr_list[-1-i]][symbol][attr]))
                
    if len(tmp) != 25:
        continue
    if files_recent[fr_list[-1]][symbol]['close'] != None or files_recent[fr_list[-1]][symbol]['close'] != 'NULL':
        Stock_feature.append(tmp)
        New_sym.append(symbol)
        Close.append(float(files_recent[fr_list[-1]][symbol]['close']))

In [24]:
print(len(Stock_feature))
print(len(New_sym))
print(len(Close))

5339
5339
5339


In [ ]:
New_sym = numpy.array(New_sym)
recent_days = numpy.array(Stock_feature)
stock_close = numpy.array(Close)
stock_open = numpy.array(Open)

numpy.save('/output/recent_days', recent_days)
numpy.save('/output/close_today', close_today)

x_predicted = numpy.load('../output/recent_days.npy')
y_today = numpy.load('../output/close_today.npy')

## 取一個月的時間作為Trainning Data

In [25]:
#-----------Stored trainning data--------------
fro = filename.index('2017-06-01.json')
to = filename.index('2017-09-30.json')

DF_list = [filename[d][:-5] for d in range(fro,to+1)]
DF = {}
for i in range(fro,to+1):
    with open('json/'+filename[i]) as f:
        symbol_f = json.load(f)
    DF[filename[i][:-5]] = symbol_f
    f.close()

In [26]:
print(DF_list[5])

2017-06-07


In [27]:
stock_feature = []
#close_td = []
#O_td = []
#C_td = []
#Open_td = []
#Close_td = []
new_sym = []
pre_close =[]

for i in range(5, len(DF_list)):
    stock_list = list(DF[DF_list[i]].keys())
    stock_list.remove('id')
    

    for stock in stock_list:
        
#        if stock not in DF[DF_list[i+1]].keys():
#            continue
            
        tmp = []   
        for j in range(0, 5):
            if stock not in DF[DF_list[i-j-1]].keys():
                break
            for attr in ['adj_close','close','high','low','open','volume']:
                if DF[DF_list[i-j-1]][stock][attr] == None or DF[DF_list[i-j-1]][stock][attr] == 'NULL':
                    break
#                if attr == 'close':
#                    C_td.append(float(DF[DF_list[i-j-1]][stock]['close']))
#                if attr == 'open':
#                    O_td.append(float(DF[DF_list[i-j-1]][stock]['open']))
                if attr != 'close':
                    tmp.append(float(DF[DF_list[i-j-1]][stock][attr]))
            
            
        if len(tmp) != 25:
            continue
            
        if DF[DF_list[i]][stock]['close'] != 'NULL' and DF[DF_list[i]][stock]['close'] != None:
            #ans = 1 if float(content[day_list[i+1]][stock]['close']) - float(content[day_list[i]][stock]['close']) > 0 else 0           
            new_sym.append(stock)
            stock_feature.append(tmp)
            pre_close.append(float(DF[DF_list[i]][stock]['close']))

In [32]:
print(pre_close[:20])

[77.95, 34.35, 27.5, 10.85, 25.45, 9.58, 10.3, 15.7, 19.25, 24.8, 32.0, 16.95, 67.5, 60.4, 7.66, 17.2, 16.8, 10.8, 80.4, 27.7]


In [28]:
print(len(pre_close))
print(len(stock_feature))
#print(len(Close_td))

304997
304997


In [45]:
new_sym = np.array(new_sym)
recent_days = np.array(stock_feature)
stock_close = np.array(pre_close)
#stock_open = numpy.array(Open)

np.save('output/recent_days.npy', recent_days)
np.save('output/close_today.npy', stock_close)

In [44]:
a = np.array([i for i in range(20)])
np.save('output/a.npy',a)

## Load Model & Predict

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline